# House Prices 데이터 전처리 실습 (연습용)

이 노트북은 Kaggle House Prices 데이터셋(`House Prices - Advanced Regression Techniques.csv`)을 이용해
결측값 처리, 깊은 복사/얕은 복사, 이상치 탐색 등을 단계적으로 연습하기 위한 실습 문제 모음입니다.

## 학습 목표
1. 결측값 탐색과 처리
2. 수치형 vs 범주형 컬럼 별로 다른 방식의 결측값 대체
3. 얕은 복사(shallow copy)와 깊은 복사(deep copy)의 차이
4. 이상치(outlier) 탐지 및 제거 (IQR 방식)
5. 전처리 파이프라인 구성 및 전/후 비교

아래 셀을 먼저 실행해서 데이터를 불러오세요.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 불러오기
df = pd.read_csv("./data/05_House Prices - Advanced Regression Techniques.csv")

df.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 문제 1️⃣: 데이터 구조 파악하기
- 데이터의 행/열 수(shape)를 확인하세요.
- 주요 수치형 변수들의 기본 통계(describe())를 확인하세요.
- info()를 통해 각 컬럼의 결측 여부와 타입을 확인하세요.

In [ ]:
# TODO: df.shape, df.describe(), df.info() 등을 이용해 데이터 구조를 살펴보세요.


## 문제 2️⃣: 결측값 개수 확인하기
- 각 컬럼별 결측값(null) 개수를 계산하고, 많은 순으로 정렬하여 상위 10개를 출력하세요.
- 어떤 컬럼들이 결측이 심한지 파악하세요.

In [ ]:
# TODO: 각 컬럼별 결측값 개수를 세고 상위 10개를 출력하세요.


## 문제 3️⃣: 결측 비율(%) 확인하기
- 각 컬럼별 결측 비율(결측 개수 / 전체 행 수 * 100)을 구한 뒤,
  비율이 높은 순서대로 정렬하여 상위 10개를 시각화(barh)하세요.
- 그래프 제목: `결측 비율 상위 컬럼`

In [ ]:
# TODO: 결측 비율 상위 10개 컬럼을 barh로 시각화하세요.


## 문제 4️⃣: 수치형 결측값 처리 (LotFrontage 예시)
- `LotFrontage`는 땅이 도로와 맞닿은 길이입니다. 이 컬럼은 결측이 존재합니다.
- 1) 원본에서 `LotFrontage`의 평균(mean)과 중앙값(median)을 계산하세요.
- 2) `LotFrontage`의 결측을 **중앙값**으로 채운 새 DataFrame `df_numfilled`를 만드세요.
- 3) 채우기 전/후의 결측 개수를 비교하세요.

In [ ]:
# TODO:
# 1) LotFrontage 평균/중앙값 계산
# 2) 중앙값으로 채운 df_numfilled 생성
# 3) 채우기 전/후 결측 개수 비교


## 문제 5️⃣: 범주형 결측값 처리 (GarageType 예시)
- `GarageType`은 차고의 형태(Attached, Detached 등)를 나타냅니다. 결측은 '차고 없음'일 가능성이 큽니다.
- 1) `GarageType`의 현재 고유값(value_counts(dropna=False))을 확인하세요.
- 2) 결측값을 문자열 `'None'`으로 채운 새 DataFrame `df_catfilled`를 만드세요.
- 3) 채우기 전/후 결측 개수를 비교하세요.

In [ ]:
# TODO:
# 1) GarageType value_counts(dropna=False)
# 2) 결측을 'None'으로 채운 df_catfilled 생성
# 3) 전/후 결측 개수 비교


## 문제 6️⃣: 전체 결측 처리 버전 만들기
- 다음 규칙으로 새로운 DataFrame `df_filled`를 만드세요.
  - 수치형 컬럼은 중앙값(median)으로 채운다.
  - 범주형 컬럼(object 타입)은 `'Unknown'`으로 채운다.
- 그런 다음 df_filled에 남아있는 결측값 개수를 모두 합산하여 출력하세요.

In [ ]:
# TODO:
# df_filled를 만들고 수치형=중앙값, 범주형='Unknown'으로 채운 뒤
# 남은 결측 총합을 출력하세요.


## 문제 7️⃣: 얕은 복사 vs 깊은 복사
- 얕은 복사(shallow copy)와 깊은 복사(deep copy)의 차이를 관찰해봅니다.
- 1) `df_shallow = df` 로 얕은 복사를 만드세요.
- 2) `df_deep = df.copy()` 로 깊은 복사를 만드세요.
- 3) 얕은 복사 쪽의 특정 컬럼(예: `LotArea`) 값을 변경했을 때 원본 df도 함께 바뀌는지 확인하세요.
- 4) 깊은 복사쪽은 독립적인지 확인하세요.

In [ ]:
# TODO:
# df_shallow = df
# df_deep = df.copy()
# df_shallow의 LotArea 첫 값을 수정하고 df/deep 비교하세요.
# (실습 후 원복도 시도해보세요.)
